# 네이버 영화 리뷰 분석

#### 글꼴 설정


In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 3s (3,552 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 128215 files and di

#### konlpy 설치


In [2]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 46.7 MB/s eta 0:00:00


#### pyLDAvis 설치


In [3]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.5 MB/s eta 0:00:00


### 관련 모듈 Import


In [4]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

# 한글 형태소 분석 모듈
from konlpy.tag import Okt
# from konlpy.tag import Mecab

from konlpy.utils import pprint

# 데이터를 받아오기 위한 모듈
import torchtext 

# 머신러닝을 위한 모듈
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression

# 시각화
import pyLDAvis.sklearn




##### 다운로드 받을 폴더를 준비

In [5]:
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

##### torchtext로 github에 있는 라벨링 데이터 다운로드 

In [6]:
train_data = torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'train.txt'))
test_data = torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', 
                                  path=os.path.join(DATA_DIR, 'test.txt'))

14.6MB [00:00, 105MB/s]
4.89MB [00:00, 103MB/s]                    


#### 구글 드라이브 마운트 후 다운받은 데이터 dataframe으로 저장

In [8]:
train_data = pd.read_csv('/content/drive/MyDrive/data/train.txt', sep='\t')
test_data = pd.read_csv('/content/drive/MyDrive/data/test.txt', sep='\t')

#### train_data와 test_data 모두 중복된 내용 제거 후 한글 데이터를 제외한 모든 문자를 공백으로 변환. 만약 공백만 있는 열은 Null값으로 변환 후 제거

In [9]:
train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거


test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


#### train_data의 리뷰 내용에서 불용어를 제거하고 okt 형태소 분석기를 활용해 형태소 단위로 나눈다.

In [10]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어 사전
okt = Okt() # 형태소 분석기
X_train = []
for sentence in tqdm(train_data['document']):

    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)
    X_train.append(stopwords_removed_sentence)

100%|██████████| 145393/145393 [05:46<00:00, 419.97it/s]


In [11]:
X_train[:5]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다',
 '교도소 이야기 구먼 솔직하다 재미 없다 평점 조정',
 '사이 몬페 그 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 커스틴 던스트 너무나도 이쁘다 보이다']

#### test_data의 리뷰 내용에서 불용어를 제거하고 okt 형태소 분석기를 활용해 형태소 단위로 나눈다.

In [12]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)
    X_test.append(stopwords_removed_sentence)

100%|██████████| 48852/48852 [01:57<00:00, 416.79it/s]


In [13]:
X_test[:5]

['굳다 ㅋ',
 '뭐 야 평점 나쁘다 않다 점 짜다 리 더 더욱 아니다',
 '지루하다 않다 완전 막장 임 돈 주다 보기 에는',
 '만 아니다 별 다섯 개 주다 왜 로 나오다 제 심기 불편하다',
 '음악 주가 되다 최고 음악 영화']

#### TF-IDF 벡터 테이블 구성. 인자에 적절한 값을 넣어줌.

*   ngram_range = (1, 3) : 1~3개의 말뭉치를 엮음.
*   min_df = 10 : 10개 미만으로 사용된 단어 삭제
*   sublinear_tf = True : 아웃라이어가 심한 값들을 조정.






In [15]:
tfid = TfidfVectorizer(ngram_range = (1, 3), min_df = 10, sublinear_tf = True)

#### X_train 형태소 데이터를 TF-IDF 벡터라이저에 학습시킨다.
######  적은 문서(문장)에 등장할수록 큰 숫자가 되게하고 반대로 많은 문서(문장)에 등장할수록 숫자를 작아지게 함으로써 여러 문서(문장)에 의미 없이 사용되는 단어의 가중치를 줄인다.

In [16]:
review_tfid = tfid.fit_transform(X_train)

In [23]:
print(review_tfid)

  (0, 18016)	0.48891677143794765
  (0, 4078)	0.5591496772599285
  (0, 6006)	0.38890441993142216
  (0, 18117)	0.3337964265267365
  (0, 17863)	0.2186883849462255
  (0, 4072)	0.37124212734037587
  (1, 234)	0.35557298335172727
  (1, 13328)	0.3703549172507712
  (1, 18471)	0.3651791958622311
  (1, 19580)	0.35557298335172727
  (1, 10774)	0.1549597170605753
  (1, 229)	0.2723462452903882
  (1, 16991)	0.2874052377964094
  (1, 12107)	0.1639388498405791
  (1, 13325)	0.30742276488972553
  (1, 12369)	0.08743385129135849
  (1, 18465)	0.26651355335906657
  (1, 7004)	0.17429386711474495
  (1, 19579)	0.2557208004859859
  (2, 7748)	0.43848025472748
  (2, 18657)	0.3110161652570861
  (2, 7115)	0.1312082089300496
  (2, 4941)	0.5235860120642707
  (2, 3593)	0.4565840464791274
  (2, 6186)	0.4595401109192198
  :	:
  (145388, 6226)	0.49047152630233826
  (145388, 17282)	0.7575999371283753
  (145389, 19446)	0.5028293630347115
  (145389, 2979)	0.48681332079234513
  (145389, 19445)	0.4571823458250066
  (145389, 2954

###### 사전 순으로 번호가 부여된 모습을 확인할 수 있다.

In [22]:
vocab = tfid.vocabulary_
print(vocab)

{'더빙': 4072, '진짜': 17863, '짜증나다': 18117, '목소리': 6006, '더빙 진짜': 4078, '진짜 짜증나다': 18016, '포스터': 19579, '보고': 7004, '초딩': 18465, '영화': 12369, '오버': 13325, '연기': 12107, '조차': 16991, '가볍다': 229, '않다': 10774, '포스터 보고': 19580, '초딩 영화': 18471, '오버 연기': 13328, '가볍다 않다': 234, '무재': 6186, '다그': 3593, '래서': 4941, '보다': 7115, '추천': 18657, '보다 추천': 7748, '교도소': 1474, '이야기': 14789, '구먼': 1509, '솔직하다': 9312, '재미': 15914, '없다': 11421, '평점': 19424, '조정': 16984, '솔직하다 재미': 9322, '재미 없다': 15940, '없다 평점': 11624, '평점 조정': 19547, '솔직하다 재미 없다': 9323, '재미 없다 평점': 15949, '사이': 8597, '돋보이다': 4224, '스파이더맨': 9689, '에서': 11749, '늙다': 3558, '보이다': 7863, '너무나도': 3183, '이쁘다': 14728, '연기 돋보이다': 12135, '돋보이다 영화': 4225, '연기 돋보이다 영화': 12136, '걸음': 1014, '떼다': 4778, '부터': 8040, '초등학교': 18460, '학년': 19882, '살다': 8650, 'ㅋㅋㅋ': 47, '반개': 6556, '아깝다': 10282, '초등학교 학년': 18462, '살다 영화': 8660, '영화 ㅋㅋㅋ': 12373, '반개 아깝다': 6557, '원작': 13814, '긴장감': 2134, '제대로': 16833, '살리다': 8668, '제대로 살리다': 16841, '나오다': 2541, '생활': 8930, '인지': 1513

##### train_data의 'label' 컬럼을 labels 변수에 저장

In [27]:
labels = train_data['label'].values

#### 로지스틱 분류 모델링 객체를 생성 

In [29]:
lr = LogisticRegression()

#### TF-IDF 벡터를 입력하여 모델 학습 

In [30]:
lr.fit(review_tfid, labels)

LogisticRegression()

#### X_test 데이터를 tf-idf 벡터로 변환 후 lr 모델에 대입해 예측

In [31]:
test_review_tfid = tfid.transform(X_test)
test_pred = lr.predict(test_review_tfid)

#### X_train 으로 학습시킨 분류 모델이 test data를 대상으로 일치한 퍼센트를 나타냄.

In [32]:
same = 0
total = 0

for test_label, pred_label in zip(test_data['label'], test_pred):
  total += 1
  # print(f'테스트 라벨: {test_label}, 예상 라벨: {pred_label} = 결과: {test_label == pred_label}')
  if test_label == pred_label:
    same += 1

print((same / total) * 100, '%')

84.47965282895275 %


### 실제 train_data를 분류모델에 학습시켜 test_data로 예측한 결과 실제 test_data의 데이터와 예측 모델의 결과의 일치율이 84% 정도가 나왔다.
